In [1]:
import pandas as pd
import os 

In [10]:
path_to_metrics_csv = 'saving_dir\\FCN-175-new\\version_0\\metrics.csv'
full_results_df = pd.read_csv(path_to_metrics_csv)
train_cols = [c for c in full_results_df.columns if 'train' in c]
val_cols = [c for c in full_results_df.columns if 'val' in c]
train_results_df = full_results_df[['epoch']+train_cols].dropna()
val_results_df = full_results_df[['epoch']+val_cols].dropna()
val_results_df

,epoch,val_iou_UNLABELED,val_iou_buildings_territory,val_iou_mean,val_iou_natural_ground,val_iou_natural_grow,val_iou_natural_wetland,val_iou_natural_wood,val_iou_quasi_natural_ground,val_iou_quasi_natural_grow,val_iou_quasi_natural_wetland,val_iou_quasi_natural_wood,val_iou_transport,val_iou_water,val_loss
0,0,0.024329,0.000000,0.032029,0.000000,0.017647,0.033727,0.075648,0.000000,0.200969,0.000000,0.0,0.000000,0.000000,2.595248
2,1,0.002273,0.017312,0.104362,0.000000,0.045519,0.028950,0.528313,0.000000,0.447091,0.000000,0.0,0.000000,0.078527,2.155669
4,2,0.274666,0.391675,0.177833,0.000000,0.058978,0.121836,0.594588,0.000000,0.430077,0.000000,0.0,0.000000,0.084345,1.611074
6,3,0.704860,0.677783,0.320623,0.000000,0.162401,0.294154,0.745156,0.000000,0.468738,0.000000,0.0,0.000000,0.473758,0.897103
8,4,0.779181,0.710217,0.313835,0.000000,0.270804,0.345013,0.768103,0.000000,0.004351,0.000000,0.0,0.000000,0.574513,0.875637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,95,0.804419,0.755872,0.420610,0.010163,0.328813,0.387071,0.794580,0.062039,0.508550,0.157360,0.0,0.129427,0.688422,1.130067
192,96,0.759080,0.751706,0.414132,0.004139,0.339938,0.378685,0.793312,0.012220,0.511662,0.183400,0.0,0.126340,0.694973,1.187167
194,97,0.787392,0.756149,0.407562,0.014220,0.253123,0.343184,0.783727,0.155914,0.509505,0.081520,0.0,0.130445,0.668001,1.200138
196,98,0.791022,0.754407,0.403821,0.000000,0.337232,0.369517,0.776061,0.044378,0.474182,0.078661,0.0,0.142840,0.673731,1.196549


# проверка правильности работы метрик

In [12]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

import numpy as np

def decode_confusion_matrix_2x2(confusion_matrix):
    tp = confusion_matrix[1, 1]
    tn = confusion_matrix[0, 0]
    fp = confusion_matrix[0, 1]
    fn = confusion_matrix[1, 0]
    return {'tp':tp, 'tn': tn, 'fp': fp, 'fn': fn}

def compute_accuracy_from_confusion(multiclass_confusion_matrix):
    confusion_sum = multiclass_confusion_matrix.sum(axis=0)
    tp, tn, fp, fn = decode_confusion_matrix_2x2(confusion_sum)
    accuracy = 0
    if tp+tn+fp+fn != 0:
        accuracy = (tp+tn)/(tp+tn+fp+fn)
    return accuracy

def compute_metric_from_confusion(multiclass_confusion_matrix, metric_params_dict, idx2class_name_dict=None):
    #print(f'metric_params_dict={metric_params_dict}')
    mean_metric = 0
    # {class_name: iou_val}
    metric_dict = {}
    actual_classes_num = 0
    metric_name = metric_params_dict['name']
    for idx, class_confusion in enumerate(multiclass_confusion_matrix):
        #print(f'class_conf_shape={class_confusion}')
        confusion_vals_dict = decode_confusion_matrix_2x2(class_confusion)
        #print(f'confusion_vals_dict={confusion_vals_dict}')
        #print(f'conf_sum={class_confusion.sum()};tn={confusion_vals_dict["tn"]}')
        if class_confusion.sum() != confusion_vals_dict['tn']:
            actual_classes_num += 1

        #print(f'actual_classes_num={actual_classes_num}')
        class_metric = 0
        numerator = [confusion_vals_dict[v] for v in metric_params_dict['numerator']]
        denominator = [confusion_vals_dict[v] for v in metric_params_dict['denominator']]
        #print(f'numerator={numerator}')
        #print(f'denominator={denominator}')
        if np.sum(denominator) != 0:
            class_metric = np.sum(numerator)/np.sum(denominator)
        mean_metric += class_metric
        class_name = f'{metric_name}_{idx}'
        if idx2class_name_dict is not None:
            class_name = f'{metric_name}_{idx2class_name_dict[idx]}'
        metric_dict[class_name] = class_metric
    if actual_classes_num == 0:
        metric_dict[f'{metric_name}_mean'] = 0
    else:
        metric_dict[f'{metric_name}_mean'] = mean_metric/actual_classes_num
    return metric_dict


In [25]:
np.random.seed(0)
class_num = 6
pred_labels = np.random.randint(0, class_num, size=[5000])
true_labels = np.random.randint(0, class_num, size=[5000])
confusion = metrics.multilabel_confusion_matrix(true_labels.reshape(-1), pred_labels.reshape(-1))
precision_results = metrics.precision_score(true_labels, pred_labels, average=None)
precision_params_dict = {'name':'precision', 'numerator':['tp'], 'denominator': ['tp', 'fp']}
precision_from_confusion = compute_metric_from_confusion(confusion,precision_params_dict,)

recall_results = metrics.recall_score(true_labels, pred_labels, average=None)
recall_params_dict = {'name':'recall', 'numerator':['tp'], 'denominator': ['tp', 'fn']}
recall_from_confusion = compute_metric_from_confusion(confusion,recall_params_dict,)

iou_results = metrics.jaccard_score(true_labels, pred_labels, average=None)
iou_params_dict = {'name':'iou', 'numerator':['tp'], 'denominator': ['tp', 'fp', 'fn']}
iou_from_confusion = compute_metric_from_confusion(confusion,iou_params_dict,)
recall_from_confusion, recall_results

({'recall_0': np.float64(0.15393939393939393),
  'recall_1': np.float64(0.15977653631284916),
  'recall_2': np.float64(0.15063291139240506),
  'recall_3': np.float64(0.16090584028605484),
  'recall_4': np.float64(0.1592814371257485),
  'recall_5': np.float64(0.16789215686274508),
  'recall_mean': np.float64(0.15873804598653277)},
 array([0.15393939, 0.15977654, 0.15063291, 0.16090584, 0.15928144,
        0.16789216]))